This notebook is adapted  from [here](http://www.science.smith.edu/~jcrouser/SDS293/labs/lab8-py.html?fbclid=IwAR0KyR4H9d8TWdsqWngLDzkI1J3Zqg7AV-uVgdqXTfiRYRz15PB1tb2p-d8). Would like to try how it looks like to choose best 10 fields from AMES dataset.

*This lab on Subset Selection is a Python adaptation of p. 244-247 of "Introduction to Statistical Learning with Applications in R" by Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani. Adapted by R. Jordan Crouser at Smith College for SDS293: Machine Learning (Spring 2016).*

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import itertools
import time
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Best Subset Selection

Here we apply the best subset selection approach to the Ames housing dataset. We
want to predict the 'SalePrice'

In [2]:
df_ames = pd.read_csv('../datasets/train_wfeature_all.csv')
df_ames.head()

,Unnamed: 0,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Lot Shape,Land Contour,...,Pool Area,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice,HouseAge,Remodel_Age,is_Remodeled,Total_SF
0,0,109,533352170,60,6.0,79.0,13517,2,3,2.0,...,0,0,3,2010,6.0,130500,34,29,0,2204.0
1,1,544,531379050,60,6.0,43.0,11492,2,3,2.0,...,0,0,4,2009,6.0,220000,13,1,1,3035.0
2,2,153,535304180,20,6.0,68.0,7922,2,4,2.0,...,0,0,1,2010,6.0,109000,57,54,0,2114.0
3,3,318,916386060,60,6.0,73.0,9802,2,4,2.0,...,0,0,4,2010,6.0,174000,4,1,1,1828.0
4,4,255,906425045,50,6.0,82.0,14235,2,3,2.0,...,0,0,3,2010,6.0,138500,110,93,0,2121.0


In [3]:
df_ames.shape

(1955, 82)

In [4]:
#define target
y = df_ames['SalePrice']

# Define the feature set X.
X = df_ames.drop(columns=['Id', 'PID', 'SalePrice'])

We can perform best subset selection by identifying the best model that contains a given number of predictors, where **best** is quantified using RSS (Residual Sum of Squares). We'll define a helper function to outputs the best set of variables for
each model size:

In [5]:
def processSubset(feature_set):
    # Fit model on feature_set and calculate RSS (Residual Sum of Squares)
    model = sm.OLS(y,X[list(feature_set)])
    regr = model.fit()
    RSS = ((regr.predict(X[list(feature_set)]) - y) ** 2).sum()
    return {"model":regr, "RSS":RSS}

In [6]:
def getBest(k):
    
    tic = time.time()
    
    results = []
    
    for combo in itertools.combinations(X.columns, k):
        results.append(processSubset(combo))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the highest RSS
    best_model = models.loc[models['RSS'].argmin()]
    
    toc = time.time()
    print("Processed", models.shape[0], "models on", k, "predictors in", (toc-tic), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

This returns a `DataFrame` containing the best model that we generated, along with some extra information about the model. Now we want to call that function for each number of predictors $k$:

In [7]:
# Could take quite awhile to complete... and my machine cannot process more than 3!

models_best = pd.DataFrame(columns=["RSS", "model"])

tic = time.time()
for i in range(1,4):
    models_best.loc[i] = getBest(i)

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

Processed 79 models on 1 predictors in 0.11440300941467285 seconds.
Processed 3081 models on 2 predictors in 4.073864936828613 seconds.
Processed 79079 models on 3 predictors in 127.92205810546875 seconds.
Total elapsed time: 134.7353639602661 seconds.


Now we have one  `DataFrame` that contains the best models we've generated along with their RSS:

In [8]:
models_best

,RSS,model
1,2.709225e+12,<statsmodels.regression.linear_model.Regressio...
2,1.712291e+12,<statsmodels.regression.linear_model.Regressio...
3,1.413885e+12,<statsmodels.regression.linear_model.Regressio...


If we want to access the details of each model, no problem! We can get a full rundown of a single model using the `summary()` function:

In [9]:
print(models_best.loc[1, "model"].summary())

                                 OLS Regression Results                                
Dep. Variable:              SalePrice   R-squared (uncentered):                   0.957
Model:                            OLS   Adj. R-squared (uncentered):              0.957
Method:                 Least Squares   F-statistic:                          4.391e+04
Date:                Mon, 31 Jan 2022   Prob (F-statistic):                        0.00
Time:                        19:49:15   Log-Likelihood:                         -23350.
No. Observations:                1955   AIC:                                  4.670e+04
Df Residuals:                    1954   BIC:                                  4.671e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

Rather than letting the results of our call to the `summary()` function print to the screen, we can access just the parts we need using the model's attributes. For example, if we want the $R^2$ value:

In [10]:
models_best.loc[3, "model"].rsquared

0.977764483210051

Excellent! In addition to the verbose output we get when we print the summary to the screen, fitting the `OLM` also produced many other useful statistics such as adjusted $R^2$, AIC, and BIC. We can examine these to try to select the best overall model. Let's start by looking at $R^2$ across all our models:

In [11]:
# Gets the second element from each row ('model') and pulls out its rsquared attribute
models_best.apply(lambda row: row[1].rsquared, axis=1)

1    0.957393
2    0.973072
3    0.977764
dtype: float64

As expected, the $R^2$ statistic increases monotonically as more
variables are included.

Forward and Backward Stepwise Selection
We can also use a similar approach to perform forward stepwise
or backward stepwise selection, using a slight modification of the functions we defined above:

In [12]:
def forward(predictors):

    # Pull out predictors we still need to process
    remaining_predictors = [p for p in X.columns if p not in predictors]
    
    tic = time.time()
    
    results = []
    
    for p in remaining_predictors:
        results.append(processSubset(predictors+[p]))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the highest RSS
    best_model = models.loc[models['RSS'].argmin()]
    
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)+1, "predictors in", (toc-tic), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

Now let's see how much faster it runs!

In [13]:
models_fwd = pd.DataFrame(columns=["RSS", "model"])

tic = time.time()
predictors = []

for i in range(1,len(X.columns)+1):    
    models_fwd.loc[i] = forward(predictors)
    predictors = models_fwd.loc[i]["model"].model.exog_names

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

Processed  79 models on 1 predictors in 0.1184239387512207 seconds.
Processed  78 models on 2 predictors in 0.1374368667602539 seconds.
Processed  77 models on 3 predictors in 0.14483308792114258 seconds.
Processed  76 models on 4 predictors in 0.157761812210083 seconds.
Processed  75 models on 5 predictors in 0.14637970924377441 seconds.
Processed  74 models on 6 predictors in 0.14864015579223633 seconds.
Processed  73 models on 7 predictors in 0.1542372703552246 seconds.
Processed  72 models on 8 predictors in 0.15868091583251953 seconds.
Processed  71 models on 9 predictors in 0.16311216354370117 seconds.
Processed  70 models on 10 predictors in 0.17609000205993652 seconds.
Processed  69 models on 11 predictors in 0.18184518814086914 seconds.
Processed  68 models on 12 predictors in 0.16529011726379395 seconds.
Processed  67 models on 13 predictors in 0.16923999786376953 seconds.
Processed  66 models on 14 predictors in 0.17212224006652832 seconds.
Processed  65 models on 15 predict

Phew! That's a lot better. Let's take a look:

In [14]:
print(models_fwd.loc[1, "model"].summary())
print(models_fwd.loc[2, "model"].summary())

                                 OLS Regression Results                                
Dep. Variable:              SalePrice   R-squared (uncentered):                   0.957
Model:                            OLS   Adj. R-squared (uncentered):              0.957
Method:                 Least Squares   F-statistic:                          4.391e+04
Date:                Mon, 31 Jan 2022   Prob (F-statistic):                        0.00
Time:                        19:49:30   Log-Likelihood:                         -23350.
No. Observations:                1955   AIC:                                  4.670e+04
Df Residuals:                    1954   BIC:                                  4.671e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

We see that using forward stepwise selection, the best one-variable
model contains only `Hits`, and the best two-variable model additionally
includes `CRBI`. Let's see how the models stack up against best subset selection:

In [15]:
#print(models_best.loc[6, "model"].summary())
print(models_fwd.loc[10, "model"].summary())

                                 OLS Regression Results                                
Dep. Variable:              SalePrice   R-squared (uncentered):                   0.986
Model:                            OLS   Adj. R-squared (uncentered):              0.986
Method:                 Least Squares   F-statistic:                          1.405e+04
Date:                Mon, 31 Jan 2022   Prob (F-statistic):                        0.00
Time:                        19:49:30   Log-Likelihood:                         -22237.
No. Observations:                1955   AIC:                                  4.449e+04
Df Residuals:                    1945   BIC:                                  4.455e+04
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------

For this data, the best one-variable through six-variable
models are each identical for best subset and forward selection.

# Backward Selection
Not much has to change to implement backward selection... just looping through the predictors in reverse!

In [16]:
def backward(predictors):
    
    tic = time.time()
    
    results = []
    
    for combo in itertools.combinations(predictors, len(predictors)-1):
        results.append(processSubset(combo))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the highest RSS
    best_model = models.loc[models['RSS'].argmin()]
    
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)-1, "predictors in", (toc-tic), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

In [17]:
models_bwd = pd.DataFrame(columns=["RSS", "model"], index = range(1,len(X.columns)))

tic = time.time()
predictors = X.columns

while(len(predictors) > 1):  
    models_bwd.loc[len(predictors)-1] = backward(predictors)
    predictors = models_bwd.loc[len(predictors)-1]["model"].model.exog_names

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

Processed  79 models on 78 predictors in 0.9147520065307617 seconds.
Processed  78 models on 77 predictors in 0.8924548625946045 seconds.
Processed  77 models on 76 predictors in 0.8668673038482666 seconds.
Processed  76 models on 75 predictors in 0.8502919673919678 seconds.
Processed  75 models on 74 predictors in 0.7982358932495117 seconds.
Processed  74 models on 73 predictors in 0.795745849609375 seconds.
Processed  73 models on 72 predictors in 0.7497646808624268 seconds.
Processed  72 models on 71 predictors in 0.725208044052124 seconds.
Processed  71 models on 70 predictors in 0.6973659992218018 seconds.
Processed  70 models on 69 predictors in 0.6804258823394775 seconds.
Processed  69 models on 68 predictors in 0.6786031723022461 seconds.
Processed  68 models on 67 predictors in 0.6806380748748779 seconds.
Processed  67 models on 66 predictors in 0.6637217998504639 seconds.
Processed  66 models on 65 predictors in 0.628868818283081 seconds.
Processed  65 models on 64 predictors

Let's check the Best 3 features for each, if they are different

In [18]:
print("------------")
print("Best Subset:")
print("------------")
print(models_best.loc[3, "model"].params)

------------
Best Subset:
------------
Overall Qual    14666.284348
HouseAge         -511.367425
Total_SF           41.371441
dtype: float64


In [19]:
print("-----------------")
print("Foward Selection:")
print("-----------------")
print(models_fwd.loc[3, "model"].params)

-----------------
Foward Selection:
-----------------
Total_SF          32.034526
Neighborhood    3026.681158
Overall Qual    8820.995273
dtype: float64


In [20]:
print("-------------------")
print("Backward Selection:")
print("-------------------")
print(models_bwd.loc[3, "model"].params)

-------------------
Backward Selection:
-------------------
Neighborhood    3026.681158
Overall Qual    8820.995273
Total_SF          32.034526
dtype: float64


All are the same for Best 3 Features. For Best 10, we can check only the backward and forward:

In [21]:
print("-----------------")
print("Foward Selection:")
print("-----------------")
print(models_fwd.loc[10, "model"].params)

-----------------
Foward Selection:
-----------------
Total_SF           38.570278
Neighborhood     1339.760030
Overall Qual     9833.153253
Land Slope     -19763.454812
Bsmt Unf SF       -18.189519
Kitchen Qual    10377.989723
Garage Area        26.766579
Fireplace Qu     3105.809960
HouseAge         -303.834379
Overall Cond     4482.438133
dtype: float64


In [22]:
print("-------------------")
print("Backward Selection:")
print("-------------------")
print(models_bwd.loc[10, "model"].params)

-------------------
Backward Selection:
-------------------
MS SubClass       -117.413648
Neighborhood      1619.774714
Overall Qual     11701.119206
Overall Cond      4933.075416
Year Built         290.345792
BsmtFin SF 1        23.490328
Total Bsmt SF      -38.031246
Kitchen Qual     11838.047131
Yr Sold           -325.104034
Total_SF            51.262003
dtype: float64


They look different. I will combine these and try the best 10 of Forward and Backward subset using the function model_best

In [23]:
# Should be quicker as we have less features than the original
#15 unique features

X = df_ames[['Total_SF', 'Neighborhood','Overall Qual','Land Slope','Bsmt Unf SF','Kitchen Qual',
            'Garage Area','Fireplace Qu','HouseAge','Overall Cond','Yr Sold','Total Bsmt SF','BsmtFin SF 1',
            'Year Built', 'MS SubClass']]
models_best = pd.DataFrame(columns=["RSS", "model"])

tic = time.time()
for i in range(1,11):
    models_best.loc[i] = getBest(i)

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

Processed 15 models on 1 predictors in 0.03155088424682617 seconds.
Processed 105 models on 2 predictors in 0.16662311553955078 seconds.
Processed 455 models on 3 predictors in 0.7221801280975342 seconds.
Processed 1365 models on 4 predictors in 2.1253740787506104 seconds.
Processed 3003 models on 5 predictors in 5.417328834533691 seconds.
Processed 5005 models on 6 predictors in 10.190083980560303 seconds.
Processed 6435 models on 7 predictors in 14.373605966567993 seconds.
Processed 6435 models on 8 predictors in 14.485986948013306 seconds.
Processed 5005 models on 9 predictors in 12.182610034942627 seconds.
Processed 3003 models on 10 predictors in 7.887772798538208 seconds.
Total elapsed time: 68.95857119560242 seconds.


In [24]:
print("------------")
print("Best Subset:")
print("------------")
print(models_best.loc[10, "model"].params)

------------
Best Subset:
------------
Total_SF           39.724766
Neighborhood     1471.621870
Overall Qual     9999.805385
Bsmt Unf SF       -19.722646
Kitchen Qual    11849.345263
Garage Area        27.314289
Fireplace Qu     2997.425554
Overall Cond     4979.112681
Yr Sold          -281.931927
Year Built        245.409920
dtype: float64
